# PA-4 Web Crawlling (19/5/6)

### 김수빈 박영준

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup  # BeautifulSoup은 class 명칭

>BeautifulSoup 라이브러리는 HTML 태그 등과 같은 Contents 를 가져와 사용자가 파싱하기 쉽게 도와주는 라이브러리

In [3]:
# 기사의 목록 가져오기
url = "http://news.donga.com/List?m="
with urlopen(url) as f:
    donga = BeautifulSoup(f, "html.parser")
donga.find_all("span", class_='tit')

[<span class="tit">英 메건 마클 왕자비, 왕자 출산…왕위 계승 서열 7위</span>,
 <span class="tit">트럼프 “대중무역적자 매년 585조원…더이상 안돼”</span>,
 <span class="tit">심재철, 유시민 합수부 진술서 공개…“학우들 행적 상세히 기록”</span>,
 <span class="tit">“‘추락사고’ 보잉, AOA센서 경보장치 결함 알고 있었다”</span>,
 <span class="tit">‘옥문아’ 임예진 “남편 나 보기 위해 PD 지원…설거지도 전담”</span>,
 <span class="tit">트럼프 배신한 집사 변호사 코언, 선고후 5개월만에 입감</span>,
 <span class="tit">나경원 “北 도발 본능 여전…美·日과 우리 정부는 달라야”</span>,
 <span class="tit">‘신라의달밤’ 작사가 유호 별세…1세대 드라마작가</span>,
 <span class="tit">국정원 “北발사체, 도발 아니다…협상 판 깨지 않겠단 의도 있어”</span>,
 <span class="tit">지동원, 마인츠05에 새 둥지 튼다</span>,
 <span class="tit">조수미, 이탈리아 훈장·기사 작위 받았다</span>,
 <span class="tit">“지구서 매년 산림 650만ha 사라져”…동식물 100만종 멸종위기</span>,
 <span class="tit">박찬주 전 대장 “지금 군은 군 아니다…육군 족보가 ‘개족보’ 돼 버려”</span>,
 <span class="tit">3기 신도시 3차 택지지구 7일 발표…광명·시흥 유력</span>,
 <span class="tit">英 ‘로열 베이비’ 탄생 둘러싼 루머에 도박업체 “출산 예정일에 베팅 금지”</span>,
 <span class="tit">‘김학의 키맨’ 윤중천, 6번째 출석…검찰, 전방위 조사</span>,
 <span class="tit">“국민인권 위한 檢개혁”…문무일, 지론 앞세워

In [4]:
# 변수 선언 및 초기화
n = 1
count = 0
store_day = 0
running = 1
tit_all = []
word_text = 'output.txt'

In [5]:
# 제목만 가져온 text를 file로 저장
def write_file(text):
    opened_file = open(word_text, 'a', encoding='UTF8')
    opened_file.write(text + '\n')
    opened_file.close()

In [13]:

while running:
    url = "http://news.donga.com/List?p=%d&prod=news&ymd=&m=" % n  # url
    with urlopen(url) as f:
        donga = BeautifulSoup(f, "html.parser")

    today = donga.find("span", class_='date')  # 오늘날짜
    today = int(today.string[8:10])
    date = donga.find_all("span", class_='date')
    date_list = []
    for i in date:
        date_list.append(int(i.string[8:10]))

    tit = donga.find_all("span", class_='tit')  # 기사 제목
    tit_list = []
    for i in tit:
        tit_list.append(i.string)  # string 으로 변경

    for i in range(len(date_list)):
        if store_day != date_list[i]:
            store_day = date_list[i]
            count += 1
            if count == 8:
                running = 0
        write_file(tit_list[i])

    n += 20
print("I'm Running {} finish!".format(n))


I'm Running 2801 finish!


In [14]:
# 제목만 저장된 파일을 열어서 읽어오기
with open(word_text, 'r', encoding='UTF8') as f:
    page = f.read()
    
words_list = page.split()  # words_list 로 page 내용 split해서 넣기

> 기사에서 가장 많이 사용된 단어를 가져오기 위해 문자열 변환 (maketrans)

In [15]:
# translate, maketrans 함수 이용해 본문 Text 불필요한 문자열 변환
in_tab = "…!\"”#$%&()*+,-./:;<=>?@[]^_`{|}~'\\"
out_tab = "                                  "
tran_tab = str.maketrans(in_tab, out_tab)  # 텍스트에 문자 변환
trans_list = []

for word in words_list:
    word = word.translate(tran_tab)
    if word.strip() == '':
        continue
    trans_list.append(word.strip())  # trans_list 에 변환된 문자 삽입

In [16]:
# list_param 에서 단어의 사용 빈도를 count 해서 dict 으로 반환
def count(list_param, dict_param):
    for wd in list_param:
        if wd in dict_param:
            dict_param[wd] += 1
    return dict_param

# trans_list 를 받아서 dictionary 로 바꿔줌
def change_dict(list_param):
    dict_wd = {}
    for wd in list_param:
        dict_wd[wd] = 0
    return dict_wd

def sort_dict(dic):  # Value 기준 sort
    import operator
    return sorted(dic.items(), key=operator.itemgetter(1), reverse=True)

In [17]:

dict_words = change_dict(trans_list)     # change_dict 으로 dictionary 형태로 만들어줌
count(trans_list, dict_words)           # 단어의 사용 빈도 수 계산
sorted_dict = sort_dict(dict_words)     # sorted 함수 이용 Value 기준으로 정렬

print(">>> 많이 사용된 단어 10개 : ", sorted_dict[:10])
for i in range(10):     # '사용 단어 - 사용된 빈도'로 출력
    print("Frequency of the word [", sorted_dict[i][0], "] is [", sorted_dict[i][1], "]")


>>> 많이 사용된 단어 10개 :  [('등', 56), ('美', 53), ('것', 52), ('첫', 51), ('文대통령', 42), ('한국당', 40), ('靑', 35), ('중', 34), ('外', 32), ('北', 31)]
Frequency of the word [ 등 ] is [ 56 ]
Frequency of the word [ 美 ] is [ 53 ]
Frequency of the word [ 것 ] is [ 52 ]
Frequency of the word [ 첫 ] is [ 51 ]
Frequency of the word [ 文대통령 ] is [ 42 ]
Frequency of the word [ 한국당 ] is [ 40 ]
Frequency of the word [ 靑 ] is [ 35 ]
Frequency of the word [ 중 ] is [ 34 ]
Frequency of the word [ 外 ] is [ 32 ]
Frequency of the word [ 北 ] is [ 31 ]


> 한자리 단어로 인해 Crawlling하는 의미가 퇴색된다고 판단하여 한자리 단어는 제외시키고 Crawlling한 결과도 첨부

In [19]:
# 한글자의 단어는 제외시키고 만든 list
except_trans_list = []

for word in words_list:
    word = word.translate(tran_tab)
    if word.strip() == '':
        continue
    if len(word) != 1:
        except_trans_list.append(word.strip())  # trans_list 에 변환된 문자 삽입

except_dict_words = change_dict(except_trans_list)     # change_dict 으로 dictionary 형태로 만들어줌
count(except_trans_list, except_dict_words)           # 단어의 사용 빈도 수 계산
except_sorted_dict = sort_dict(except_dict_words)     # sorted 함수 이용 Value 기준으로 정렬

print(">>> 많이 사용된 단어 10개 : ", except_sorted_dict[:10])
for i in range(10):     # '사용 단어 - 사용된 빈도'로 출력
    print("Frequency of the word [", except_sorted_dict[i][0], "] is [", except_sorted_dict[i][1], "]")


>>> 많이 사용된 단어 10개 :  [('것', 45), ('文대통령', 42), ('한국당', 40), ('만에', 31), ('vs', 28), ('경찰', 27), ('황교안', 26), ('검찰', 25), ('5월', 24), ('패스트트랙', 24)]
Frequency of the word [ 것 ] is [ 45 ]
Frequency of the word [ 文대통령 ] is [ 42 ]
Frequency of the word [ 한국당 ] is [ 40 ]
Frequency of the word [ 만에 ] is [ 31 ]
Frequency of the word [ vs ] is [ 28 ]
Frequency of the word [ 경찰 ] is [ 27 ]
Frequency of the word [ 황교안 ] is [ 26 ]
Frequency of the word [ 검찰 ] is [ 25 ]
Frequency of the word [ 5월 ] is [ 24 ]
Frequency of the word [ 패스트트랙 ] is [ 24 ]


> 한 글자를 제외시킨 결과를 가져와 조금 더 의미 있는 Crawlling 결과를 나타낼 수 있었습니다.